In [1]:
from copy import deepcopy
from radd.toolbox.theta import get_optimized_params
from radd import fit, models, build
from radd.toolbox.analyze import ensure_numerical_wts
from radd.toolbox.messages import logger
from numpy import array
from radd import models
from lmfit import Parameters, minimize
from radd import fit

# Build Model & Optimizer Objects

In [23]:
os.chdir('/Users/kyle/Dropbox/TEST_OLD_FITS/')

data=prodf_store.prodata
depends_on = {'v':'pGo'}
kind = 'xpro'

m=build.Model(data=data, depends_on=depends_on, kind=kind, split=50, include_zero_rts=0)
fp = m.set_fitparams(tol=.00001, get_params=True)

m.__check_inits__()

inits = dict(deepcopy(m.inits))

opt = fit.Optimizer(dframes=m.dframes, fitparams=fp, kind=m.kind, inits=inits, depends_on=m.depends_on, fit_on=m.fit_on, wts=m.wts, pc_map=m.pc_map)


      Model is prepared to fit xpro model (w/ hyp dynamic bias) to average data,
      allowing Drift-Rate to vary across levels of pGo (0, 20, 40, 60, 80, 100)

      See, it's not that simplex... 



# Setup Model/Parameters

In [3]:
inits

{'a': 0.3267, 'tr': 0.3192, 'v': 1.3813, 'xb': 0.09, 'z': 0}

In [24]:
pnames = deepcopy(opt.pnames)
pfit = list(set(m.inits.keys()).intersection(pnames))
lim = set_bounds(dynamic=m.dynamic)
fp = m.fitparams
fp['tol']=.0001
fp['maxfev']=1000
simulator.ntrials=2000
is_flat = False

ip = deepcopy(inits)
lmParams=Parameters()

for pkey, pc_list in m.pc_map.items():
      if is_flat: 
            break
      else:
            pfit.remove(pkey)
            #bump = np.linspace(.8,1.20,opt.ncond)
            bump=np.ones(opt.ncond)
            if pkey in ['a', 'tr']:                        
                  bump = bump[::-1]
            vals = ip[pkey]*bump
            
      mn = lim[pkey][0]; mx=lim[pkey][1]
      d0 = [lmParams.add(pc, value=vals[i], vary=1, min=mn, max=mx) for i, pc in enumerate(pc_list)]

p0 = [lmParams.add(k, value=ip[k], vary=is_flat, min=lim[k][0], max=lim[k][1]) for k in pfit]
opt_kws = {'disp':fp['disp'], 'xtol':fp['tol'], 'ftol':fp['tol'], 'maxfev':fp['maxfev']}

simulator = models.Simulator(model=m, inits=lmParams.valuesdict())
if is_flat:
      simulator.wts = fp['fwts']
      simulator.y = fp['flat_y']
else:
      simulator.wts = m.wts
      simulator.y = m.avg_y
      

# Fit Model

In [500]:
def cost_fx_local(p):
      yhat = simulator.sim_fx(p.valuesdict(), analyze=True)
      cost = (yhat - simulator.y)*simulator.wts
      return cost

In [37]:
optmod = minimize(simulator.__cost_fx__, lmParams, method='nelder')#, options=opt_kws)
# OPTIMIZE THETA
optp = optmod.params
finfo = {k:optp[k].value for k in optp.keys()}
popt = deepcopy(finfo)
residual = optmod.residual
yhat = simulator.y + residual
wts = simulator.wts
log_arrays = {'y': simulator.y, 'yhat':yhat, 'wts':wts}
optmod.unprepare_fit()
logger(optmod=optmod, finfo=finfo, depends_on=fp['depends_on'], log_arrays=log_arrays, kind=m.kind, fit_id='test tr')

In [39]:
m.inits

{'a': 0.4748, 'tr': 0.2725, 'v': 1.6961, 'xb': 0.09, 'z': 0}

In [38]:
popt

{'a': 0.4748,
 'tr': 0.2725,
 'v_0': 1.6961038131318127,
 'v_100': 1.6960978437877634,
 'v_20': 1.6810264973294495,
 'v_40': 1.6960977748528194,
 'v_60': 1.6960989321922046,
 'v_80': 1.6961006664018536,
 'xb': 0.09,
 'z': 0.001}

In [5]:
def set_bounds(a=(.001, 1.000), tr=(.01, .55), v=(.0001, 4.0000), z=(.001, .900), ssv=(-4.000, -.0001), xb=(.01,10), si=(.001, .2), dynamic='hyp'):

      """ set and return boundaries to limit search space
      of parameter optimization in <optimize_theta>
      """

      if dynamic == 'exp':
            xb = (.01, 10)
      elif dynamic == 'hyp':
            #xb = (.001, .1)
            xb = (.01, 10)

      if 'irace' in kind:
            ssv=(abs(ssv[1]), abs(ssv[0]))

      bounds = {'a': a, 'tr': tr, 'v': v, 'ssv': ssv, 'z': z, 'xb':xb, 'si':si}
      return bounds


In [31]:
p = m.sim.vectorize_params(p)

In [309]:
m.opt.lmParams

Parameters([('tr_0', <Parameter 'tr_0', 0.32008138204308784, bounds=[0.001:0.55]>), ('tr_20', <Parameter 'tr_20', 0.32008138204308784, bounds=[0.001:0.55]>), ('tr_40', <Parameter 'tr_40', 0.32008138204308784, bounds=[0.001:0.55]>), ('tr_60', <Parameter 'tr_60', 0.32008138204308784, bounds=[0.001:0.55]>), ('tr_80', <Parameter 'tr_80', 0.32008138204308784, bounds=[0.001:0.55]>), ('tr_100', <Parameter 'tr_100', 0.32008138204308784, bounds=[0.001:0.55]>), ('a', <Parameter 'a', value=0.32511466588689114 (fixed), bounds=[0.001:1.0]>), ('z', <Parameter 'z', value=0.0010029295479902526 (fixed), bounds=[0.001:0.9]>), ('v', <Parameter 'v', value=1.3800721260201678 (fixed), bounds=[0.0001:4.0]>)])

In [498]:
p = simulator.vectorize_params(p)
t = np.cumsum([.0005]*600)
x = simulator.temporal_dynamics(p, t)
plt.plot(x[0])

In [50]:
ex = np.exp(2.5*t)
hy = np.cosh(2.5*t)